# Hotelreservierungssystem

## Ausführen der User Stories

Folgend wird das Menu des [input_helper.py](https://deepnote.com/workspace/BAI-Projekte-8a9d47a8-bcd7-44ff-8444-0996c6ccb0b9/project/AEP-Hotelreservierungsysstem-Team-B8-a048451d-c7e6-46c3-a824-c0d893d5e1b2/Hotelreservierungssystem%2Fui_folder%2Finput_helper.py?utm_content=a048451d-c7e6-46c3-a824-c0d893d5e1b2) File ausgeführt. Dadurch öffnet sich in der Kommandozeile ein Menu in dem man zwischen Gast oder Admin auswählen kann. Hat man eines ausgewählt, findet man die entsprechenden User Stories.

In [1]:
import sys
sys.path.append('Hotelreservierungssystem/')
from ui_folder import input_helper

input_helper.main_menu()

FileNotFoundError: [Errno 2] No such file or directory: 'database/hotel_reservation_db.db'

## Dokumentation Minimale User Stories

Die folgenden User Stories dienen nicht zur Ausführung, sondern nur als Darstellung für den Aufbau. Wenn man die User Stories testen will, muss man das main.menu() ausführen.

### User Stories 1

#### User Story 1.1
##### "*Ich möchte alle Hotels in einer Stadt durchsuchen, damit ich das Hotel nach meinem bevorzugten Standort (Stadt) auswählen kann.*"

##### Beteiligte Komponenten
	•	ui_folder/input_helper.py → Eingabe und Ausgabe
	•	business_logic/hotel_manager.py → Business-Logik
	•	data_access/hotel_dal.py → Datenbankzugriff für Hotels
	•	data_access/address_dal.py → Datenbankzugriff auf Adressen
	•	model/hotel.py, model/address.py → Datenmodelle

##### Ablauf der User Story
    1.	Der Nutzer gibt im Menü „Hotels nach Stadt filtern“ eine Stadt ein.
	2.	Die Eingabe wird an den HotelManager weitergeleitet.
	3.	HotelManager ruft alle Hotels über HotelDAL ab.
	4.	Die zugehörige Adresse jedes Hotels wird mit AddressDAL geladen.
	5.	Hotels, deren Stadt zur Eingabe passt, werden zurückgegeben.
	6.	Ausgabe erfolgt mit Hotelname, Adresse und Sternen.

##### Wichtiger Code


a) input_helper.py

In [ ]:
def user_story_1_1():
    # Print the title of the feature
    print("\n--- 1.1: Hotels nach Stadt filtern ---")
    # Ask the user to enter a city name
    city = input("Stadt eingeben: ")
    # Get hotels in the entered city using HotelManager
    hotels = HotelManager().filter_by_city(city)
    # Print info about each hotel
    for hotel in hotels:
        print(f"ID: {hotel.hotel_id} | Name: {hotel.name} | Sterne: {hotel.stars}")

b) hotel_manager.py

In [ ]:
def filter_by_city(self, city: str) -> list[Hotel]:
    # get all hotels from the database
    hotels = self.hotel_dal.get_all_hotels()
    # return only hotels where the city matches
    return [h for h in hotels if self.address_dal.get_address_by_hotel(h.hotel_id).city.lower() == city.lower()]

c) address_dal.py

In [ ]:
def get_address_by_hotel(self, hotel_id: int) -> Address | None:
    # run SQL query to get address for a given hotel ID using a JOIN
    cursor = self.conn.execute(
        "SELECT a.* FROM address a JOIN hotel h ON a.address_id = h.address_id WHERE h.hotel_id = ?",
        (hotel_id,)
    )
    # get the first result row
    row = cursor.fetchone()
    # if a row was found, create and return an Address object using keyword args; else return None
    return Address(**row) if row else None

d) hotel_dal.py

In [ ]:
def get_all_hotels(self) -> list[Hotel]:
    # SQL query to get all hotels with their basic info
    sql = "SELECT hotel_id, name, stars, address_id FROM Hotel"
    # get all rows from the database
    rows = self.fetchall(sql)
    # create a list of Hotel objects from the rows
    return [Hotel(hotel_id=row[0], name=row[1], stars=row[2], address_id=row[3]) for row in rows]

##### Testen der Funktion
In der UI startet man das Programm und wählt:

1 (Gast) → 1.1 → Eingabe: "Zürich"
Es erscheint eine Liste der Hotels in Zürich mit ID, Name und Sternen: 
```python 
ID: 1 | Name: Hotel Baur au Lac | Sterne: 5
```

##### Fehlerbehandlung

- Ungültige oder leere Eingaben
- Städte mit keinen passenden Hotels

#### User Story 1.2
##### "*Ich möchte alle Hotels in einer Stadt nach der Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.*"

##### Beteiligte Komponenten
	•	ui_folder/input_helper.py → Eingabe und Ausgabe
	•	business_logic/hotel_manager.py → Business-Logik
	•	data_access/hotel_dal.py → Datenbankzugriff für Hotels
	•	data_access/address_dal.py → Zugriff auf Adressdaten
	•	model/hotel.py, model/address.py → Datenmodelle

##### Ablauf der User Story
	1.	Der Nutzer wählt im Menü "Hotels nach Sternen in Stadt filtern" aus.
	2.	Es werden die Stadt und die minimale Sterneanzahl abgefragt.
	3.	Die Eingaben werden an den HotelManager weitergeleitet.
	4.	HotelManager lädt alle Hotels der Stadt über die bestehende Filterfunktion.
	5.	Danach werden Hotels gefiltert, die mindestens die gewünschte Sterneanzahl haben.
	6.	Die passenden Hotels werden mit ID, Name und Sternen ausgegeben.

##### Wichtiger Code


a) input_helper.py

In [ ]:
def user_story_1_2():
    # print the title for this user story
    print("\n--- 1.2: Hotels nach Sternen in Stadt filtern ---")
    # ask user to enter a city
    city = input("Stadt eingeben: ")
    # ask user to enter the minimum number of stars
    min_stars = int(input("Minimale Sterne (1–5): "))
    # get hotels that match the city and have at least the given stars
    hotels = HotelManager().filter_by_city_and_stars(city, min_stars)
    # print info about each hotel
    for hotel in hotels:
        print(f"ID: {hotel.hotel_id} | Name: {hotel.name} | Sterne: {hotel.stars}")

b) hotel_manager.py

In [ ]:
def filter_by_city_and_stars(self, city: str, min_stars:
    # get hotels in the given city
    hotels = self.filter_by_city(city)
    # return only hotels with stars >= min_stars
    return [h for h in hotels if h.stars >= min_stars]

c) address_dal.py

In [ ]:
def get_address_by_hotel(self, hotel_id: int) -> Address | None:
    # run SQL query to get address for a given hotel ID using a JOIN
    cursor = self.conn.execute(
        "SELECT a.* FROM address a JOIN hotel h ON a.address_id = h.address_id WHERE h.hotel_id = ?",
        (hotel_id,)
    )
    # get the first result row
    row = cursor.fetchone()
    # if a row was found, create and return an Address object using keyword args; else return None
    return Address(**row) if row else None

d) hotel_dal.py

In [ ]:
def get_all_hotels(self) -> list[Hotel]:
    # SQL query to get all hotels with their basic info
    sql = "SELECT hotel_id, name, stars, address_id FROM Hotel"
    # get all rows from the database
    rows = self.fetchall(sql)
    # create a list of Hotel objects from the rows
    return [Hotel(hotel_id=row[0], name=row[1], stars=row[2], address_id=row[3]) for row in rows]

##### Testen der Funktion
In der UI startet man das Programm und wählt:

1 (Gast) → 1.2 → Eingabe: "Zürich", "4"
Es erscheint eine Liste der Hotels in Zürich mit ID, Name und Sternen: 
```python 
ID: 1 | Name: Hotel Baur au Lac | Stars: 5
ID: 3 | Name: The Dolder Grand | Stars: 5
```

##### Fehlerbehandlung

- Eingabe ist keine Zahl zwischen 1-5 → Exception oder Hinweis anzeigen
- Stadt existiert nicht → leere Liste
- Keine Hotels mit ausreichend Sternen → Hinweis ausgeben

#### User Story 1.3
##### "*Ich möchte alle Hotels in einer Stadt durchsuchen, die Zimmer haben, die meiner Gästezahl entsprechen (nur 1 Zimmer pro Buchung).*"

##### Beteiligte Komponenten
	•	ui_folder/input_helper.py → Eingabe und Ausgabe
	•	business_logic/hotel_manager.py → Business-Logik
	•	data_access/hotel_dal.py → Zugriff auf Hoteldaten
	•	data_access/address_dal.py → Zugriff auf Adressdaten
	•	data_access/room_dal.py → Zugriff auf Zimmerdaten
	•	model/hotel.py, model/address.py, model/room.py → Datenmodelle

##### Ablauf der User Story
	1.	Der Nutzer gibt im Menü „Hotels mit passenden Zimmern für Gästezahl“ eine Stadt und eine Gästezahl ein.
	2.	Die Eingaben werden an den HotelManager übergeben.
	3.	HotelManager ruft alle Hotels über HotelDAL ab.
	4.	Die Adresse jedes Hotels wird mit AddressDAL geladen, um die Stadt zu überprüfen.
	5.	Zimmer des Hotels werden mit RoomDAL geladen.
	6.	Hotels, die mindestens ein Zimmer mit ausreichender Kapazität haben, werden zurückgegeben.
	7.	Ausgabe erfolgt mit Hotelname, ID und Sternebewertung.

##### Wichtiger Code

a) input_helper.py

In [ ]:
def user_story_1_3():
    # print the title for this user story
    print("\n--- 1.3: Hotels mit passenden Zimmern für Gästezahl in Stadt ---")
    # ask user for a city
    city = input("Stadt: ")
    # ask user for number of guests
    guests = int(input("Anzahl Gäste: "))
    # get hotels in the city that have rooms for the given number of guests
    hotels = HotelManager().filter_by_city_and_guest_capacity(city, guests)
    # print hotel name and stars for each result
    for hotel in hotels:
        print(f"Hotel: {hotel.name} | Sterne: {hotel.stars}")

b) hotel_manager.py

In [ ]:
def filter_by_city_and_guest_capacity(self, city: str, guests: int) -> list[Hotel]:
    # get hotels in the given city
    hotels = self.filter_by_city(city)
    # list to store hotels that have rooms for the guests
    matching_hotels = []
    # check each hotel
    for hotel in hotels:
        # get all rooms for the hotel
        rooms = self.room_dal.get_rooms_by_hotel_id(hotel.hotel_id)
        # check each room
        for room in rooms:
            # get room type to check max guests
            room_type = self.room_type_dal.get_by_id(room.type_id)
            # if room can fit the number of guests
            if room_type.max_guests >= guests:
                # add hotel to result and stop checking more rooms
                matching_hotels.append(hotel)
                break
    # return hotels with suitable rooms
    return matching_hotels

c) address_dal.py

In [ ]:
def get_address_by_hotel(self, hotel_id: int) -> Address | None:
    # run SQL query to get address for a given hotel ID using a JOIN
    cursor = self.conn.execute(
        "SELECT a.* FROM address a JOIN hotel h ON a.address_id = h.address_id WHERE h.hotel_id = ?",
        (hotel_id,)
    )
    # get the first result row
    row = cursor.fetchone()
    # if a row was found, create and return an Address object using keyword args; else return None
    return Address(**row) if row else None

d) hotel_dal.py

In [ ]:
def get_all_hotels(self) -> list[Hotel]:
    # SQL query to get all hotels with their basic info
    sql = "SELECT hotel_id, name, stars, address_id FROM Hotel"
    # get all rows from the database
    rows = self.fetchall(sql)
    # create a list of Hotel objects from the rows
    return [Hotel(hotel_id=row[0], name=row[1], stars=row[2], address_id=row[3]) for row in rows]

e) room_dal.py

In [ ]:
def get_rooms_by_hotel_id(self, hotel_id: int) -> list[Room]:
    # SQL query to get all rooms for the given hotel ID
    sql = "SELECT * FROM room WHERE hotel_id = ?"
    # get all matching rows from the database
    rows = self.fetchall(sql, (hotel_id,))
    # create a list of Room objects from the rows
    return [
        Room(
            room_id=row[0],
            hotel_id=row[1],
            room_no=row[2],
            type_id=row[3],
            price_per_night=row[4]
        )
        for row in rows
    ]

##### Testen der Funktion
In der UI startet man das Programm und wählt:

1 (Gast) → 1.3 → Eingabe: "Zürich", Gäste: 2
Es erscheint eine Liste der Hotels in Zürich mit ID, Name und Sternen: 
```python 
ID: 2 | Name: Hotel Central Plaza | Sterne: 4
ID: 5 | Name: Hotel Opera Zürich | Sterne: 3
```

##### Fehlerbehandlung

- Nicht-numerische oder ungültige Gästeanzahl → Abbruch mit Hinweis
- Keine passenden Hotels → Ausgabe: „Keine passenden Hotels gefunden.“

### User Stories 2

### User Stories 3

### User Story 4

### User Story 5

### User Story 6

### User Story 7

### User Story 8

### User Story 9

### User Story 10

## User Stories mit DB-Schemaänderung

## User Stories mit Datenvisualisierung

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a048451d-c7e6-46c3-a824-c0d893d5e1b2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>